In [16]:
import sys

import tensorflow as tf
import pandas as pd
import numpy as np

import yaml

sys.path.append("/home/thomas/projects/nilm_at_fair/repository")
from src.pulsed_power_ml.model_framework.data_io import read_training_files
from src.pulsed_power_ml.model_framework.visualizations import plot_data_point_array
from src.pulsed_power_ml.model_framework.visualizations import plot_state_vector_array

from src.pulsed_power_ml.models.gupta_model.tf_gupta_clf import TFGuptaClassifier

from src.pulsed_power_ml.models.gupta_model.gupta_clf import GuptaClassifier

print(sys.executable)

/home/thomas/.venv/py10_kitchensink/bin/python3.10


# Load training and validation data

In [17]:
# Load training data
# training_data_folder = "/home/thomas/projects/nilm_at_fair/training_data/more_labels_20221116/"
training_data_folder = "/home/thomas/projects/nilm_at_fair/training_data/training_data_maria/fair/labels_20221202_9peaks/"

features_file = f"{training_data_folder}/Features_ApparentPower_0.7_p.csv"
labels_file = f"{training_data_folder}/Labels_ApparentPower_0.7_p.csv"

features = tf.constant(value=pd.read_csv(features_file).values, dtype=tf.float32)
labels = tf.constant(value=pd.read_csv(labels_file).values, dtype=tf.float32)

val_data_folder = "/home/thomas/projects/nilm_at_fair/training_data/training_data_maria/fair/labels_20221202_9peaks_validation/"

features_val_file = f"{val_data_folder}/Features_ApparentPower_0.3_p.csv"
labels_val_file = f"{val_data_folder}/Labels_ApparentPower_0.3_p.csv"

features_val = tf.constant(value=pd.read_csv(features_val_file).values, dtype=tf.float32)
labels_val = tf.constant(value=pd.read_csv(labels_val_file).values, dtype=tf.float32)

In [18]:
labels.shape

TensorShape([306, 23])

In [19]:
# load apparent power list
apparent_power_data_base = "/home/thomas/projects/nilm_at_fair/repository/src/pulsed_power_ml/models/gupta_model/apparent_power_data_base.yml"

with open(apparent_power_data_base) as yaml_file:
    apparent_power_dict = yaml.load(yaml_file, Loader=yaml.loader.SafeLoader)

appliance_names = apparent_power_dict.keys()

apparent_power_list = tf.constant(
    value=list(apparent_power_dict.values()),
    dtype=tf.float32
)

In [20]:
# Load some GNU Radio data for simulated "live data"
gr_data_folder = "/home/thomas/projects/nilm_at_fair/training_data/2022-11-16_training_data/r1"
data_point_array = read_training_files(path_to_folder=gr_data_folder,
                                       fft_size=2**17)

# Instantiate TensorFlow Model

In [21]:
tf_model = TFGuptaClassifier(
    background_n=tf.constant(2, dtype=tf.int32),
    n_known_appliances=tf.constant(11, dtype=tf.int32),
    apparent_power_list=apparent_power_list,
    n_neighbors=tf.constant(3, dtype=tf.int32),
    training_data_features=features,
    training_data_labels=labels
)

In [22]:
state_vector_list = list()
for data_point in data_point_array:
    state_vector = tf_model.predict(tf.constant(data_point, dtype=tf.float32))
    state_vector_list.append(state_vector)

[[198.364258]
 [228.881836]
 [259.399414]] [6253.84131 6429.42773 3003.44824]
[[381.469727]
 [411.987305]
 [442.504883]] [2681.84424 4241.05664 3092.94653]
[[595.092773]
 [625.610352]
 [656.12793]] [3288.5188 3655.90601 1855.35303]
[[778.198242]
 [808.71582]
 [839.233398]] [2217.94263 3797.97 2985.16]
[[991.821289]
 [1022.33887]
 [1052.85645]] [2313.25024 2767.98657 1516.67847]
[[1174.92676]
 [1205.44434]
 [1235.96191]] [1216.54993 2238.62109 1877.79077]
[[1388.5498]
 [1419.06738]
 [1449.58496]] [1515.07141 1939.6543 1129.22791]
[[1571.65527]
 [1602.17285]
 [1632.69043]] [584.720093 1149.04675 1023.27332]
[[1785.27832]
 [1815.7959]
 [1846.31348]] [696.658569 943.683167 578.747437]


InvalidArgumentError: Exception encountered when calling layer 'TFGuptaClassifier' (type TFGuptaClassifier).

Graph execution error:

Detected at node 'div_no_nan' defined at (most recent call last):
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/thomas/.venv/py10_kitchensink/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/thomas/.venv/py10_kitchensink/lib/python3.10/site-packages/traitlets/config/application.py", line 985, in launch_instance
      app.start()
    File "/home/thomas/.venv/py10_kitchensink/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/home/thomas/.venv/py10_kitchensink/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start
      

In [ ]:
fig = plot_state_vector_array(state_vector_list=np.array(state_vector_list),
                              label_list=list(appliance_names),
                              true_apparent_power=data_point_array[:,-2])
fig.savefig=("/home/thomas/projects/nilm_at_fair/training_data/model_validation/tf_model_r1_plot.pdf")

# Instantiate Python model

In [ ]:
py_app_power_list = list()
for app_power in apparent_power_list:
    py_app_power_list.append(("app_name", app_power))

py_model = GuptaClassifier(
    background_n=2,
    spectrum_type=2,
    apparent_power_list=py_app_power_list,
    n_neighbors=3,
    n_peaks_max=9,
    distance_threshold=1000,
    n_known_appliances=11,
)

In [ ]:
_ = py_model.fit(features,
                 labels)

In [ ]:
state_vector_list_py = list()
for data_point in data_point_array:
    state_vector = py_model.predict(data_point)
    state_vector_list_py.append(state_vector)

In [ ]:
# fig = plot_data_point_array(data_point_array,
#                             2**17,
#                             state_vector_list_py)
# fig.savefig("/home/thomas/projects/nilm_at_fair/training_data/model_validation/py_model_fp1_no_spectra.pdf")

In [ ]:
fig = plot_state_vector_array(state_vector_list=np.array(state_vector_list_py),
                              label_list=list(appliance_names),
                              true_apparent_power=data_point_array[:,-2])
fig.savefig=("/home/thomas/projects/nilm_at_fair/training_data/model_validation/py_model_r1_alt_plot.pdf")